# AI Action Plan Comments Report

BMZA | May 13, 2025

**This analysis examines public comments submitted in response to the National AI Action Plan, with a specific focus on those referencing or relevant to the U.S. Department of Commerce.** 

The project leverages the AI-enhanced dataset provided by IFP[^2], which uses Claude (claude-3-7-sonnet-latest) to topic cluster, extract recommendations, and summarize across thousands of public submissions. To focus on substantive policy input, IFP filtered out non-substantive comments—primarily brief, anonymous, or pseudonymous submissions lacking detailed policy recommendations.[^3] This filtering process resulted in a curated set of 721 submissions from organizations, which were more likely to contain meaningful policy ideas. 

By filtering and analyzing content related to the Department of Commerce, this work seeks to identify key public concerns, priorities, and recommendations regarding the Department’s role in AI governance, research, and regulation.

[^2]: [IFP AI Action Plan Database](aiactionplan.org)
[^3]: [IFP AI Action Plan Analysis](https://ifp.org/ai-action-plan/)

## Background and Context

In February 2025, the White House Office of Science and Technology Policy (OSTP) issued a Request for Information (RFI) to solicit public input on the development of a national AI Action Plan.[^1] This initiative was launched under President Trump’s Executive Order on Removing Barriers to American Leadership in AI, aiming to gather perspectives from a wide range of stakeholders—including academia, industry, civil society, and various levels of government—on AI policy priorities. Over a seven-week period, more than 10,000 comments were submitted, covering topics such as semiconductor manufacturing, supply chain resilience, AI model development, workforce training, and scientific research. The comment period closed on March 15, 2025, and the full catalog of submissions was published by OSTP on April 24, 2025. [^2]

The Department of Commerce plays a pivotal role in the federal AI ecosystem through its bureaus, including the National Institute of Standards and Technology (NIST) and the U.S. Patent and Trademark Office (USPTO). These bureaus are central to the development of technical standards, intellectual property frameworks, and innovation policy related to AI. Analyzing public feedback directed toward or implicating the Department of Commerce provides valuable insights into stakeholder expectations and concerns, thereby informing the Department’s ongoing efforts in AI governance and policy formulation.

[^1]: [Request for Information on the Development of an Artificial Intelligence (AI) Action Plan](https://www.federalregister.gov/documents/2025/02/06/2025-02305/request-for-information-on-the-development-of-an-artificial-intelligence-ai-action-plan)
[^2]: [The White House: American Public Submits Over 10,000 Comments on White House’s AI Action Plan](https://www.whitehouse.gov/articles/2025/04/american-public-submits-over-10000-comments-on-white-houses-ai-action-plan/)


In [12]:
import pandas as pd
import nbconvert

# Importing data from aiactionplan.org
data = pd.read_csv('recommendations.csv')

# Extracting Commerce assignees
assignees = [
  "All",
  "NIST",
  "Department of Commerce",
  "AISI",
  "Bureau of Industry and Security (BIS)",
  "USPTO",
  "National Semiconductor Technology Center",
  "NOAA",
  "Census Bureau",
  "USTR",
  "NAIC",
  "CHIPS Program Office",
  "Manufacturing USA",
  "PTO",
  "SelectUSA",
  "Patent and Trial Appeal Board",
  "Foreign Commercial Service",
  "NCI",
  "EDA",
  "CHIPS Manufacturing USA",
  "End-User Review Committee",
  "Bureau of Economic Analysis"
]
commerce_recommendations = data[data['Assignee'].isin(assignees)]
commerce_recommendations.to_excel('commerce_recommendations_ai_action_plan_comments.xlsx')

## Breakdown by Organization Type

In [32]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from IPython.display import display


pio.renderers.default = 'notebook_connected'

# Group counts 
overall_counts = data['OrgType'].value_counts().reset_index()
overall_counts.columns = ['OrgType', 'Overall Count']

commerce_counts = commerce_recommendations['OrgType'].value_counts().reset_index()
commerce_counts.columns = ['OrgType', 'Commerce Count']

# Merge and calculate proportions 
merged = pd.merge(overall_counts, commerce_counts, on='OrgType', how='outer').fillna(0)
merged['Overall Proportion'] = merged['Overall Count'] / merged['Overall Count'].sum()
merged['Commerce Proportion'] = merged['Commerce Count'] / merged['Commerce Count'].sum()

# Build figure and traces 
fig = go.Figure()

# Traces
fig.add_trace(go.Bar(
  x=merged['OrgType'],
  y=merged['Overall Count'],
  name='Overall (Count)',
  visible=True
))
fig.add_trace(go.Bar(
  x=merged['OrgType'],
  y=merged['Commerce Count'],
  name='Commerce (Count)',
  visible=True
))
fig.add_trace(go.Bar(
  x=merged['OrgType'],
  y=merged['Overall Proportion'],
  name='Overall (Proportion)',
  visible=False
))
fig.add_trace(go.Bar(
  x=merged['OrgType'],
  y=merged['Commerce Proportion'],
  name='Commerce (Proportion)',
  visible=False
))

# Add dropdown with legend visibility toggled 
fig.update_layout(
  updatemenus=[
   {
    'buttons': [
      {
       'label': 'Counts',
       'method': 'update',
       'args': [
        {'visible': [True, True, False, False]},
        {
          'yaxis': {'title': 'Number of Recommendations'},
          'legend': {'title': {'text': 'Legend'}}
        }
       ]
      },
      {
       'label': 'Proportions',
       'method': 'update',
       'args': [
        {'visible': [False, False, True, True]},
        {
          'yaxis': {'title': 'Proportion of Recommendations'},
          'legend': {'title': {'text': 'Legend'}}
        }
       ]
      }
    ],
    'direction': 'down',
    'showactive': True
   }
  ],
  barmode='group',
  title='Organization Types: Counts vs. Proportions',
  xaxis_title='Organization Type',
  yaxis_title='Number of Recommendations',
  template='ggplot2'
)

display(fig)


The subset of recommendations directed at the Department of Commerce reveals a notable overrepresentation of industry associations and think tanks, suggesting that the comments assigned to Commerce are more influenced by organized industry coalitions and policy-oriented groups than by technical or research-based communities. This aligns with the Department's focus on export controls, standards, innovation policy, and economic competitiveness, areas where industry groups typically prioritize regulatory clarity and market dynamics. In contrast, academic institutions and professional societies—critical sources of subject-matter expertise—are underrepresented in Commerce-specific recommendations compared to the overall comment pool. This imbalance highlights the risk of underweighting academic and technical perspectives, which are essential for assessing the feasibility, risks, and long-term implications of AI development.

## Recommendations by topic, stacked by bureau

In [60]:
exploded_df = commerce_recommendations.copy()
exploded_df['Topic'] = exploded_df['Topic'].str.split(',')

# Remove leading/trailing whitespace from topics
exploded_df = exploded_df.explode('Topic')
exploded_df['Topic'] = exploded_df['Topic'].str.strip()

# Group by cleaned Topic and Assignee, then count
grouped = exploded_df.groupby(['Topic', 'Assignee']).size().reset_index(name='Count')

# Pivot the table to get one row per Topic
topic_wide = grouped.pivot(index='Topic', columns='Assignee', values='Count').fillna(0).astype(int).reset_index()

# Sort by total count across all assignees
topic_wide['Total'] = topic_wide.drop(columns='Topic').sum(axis=1)
topic_wide = topic_wide.sort_values(by='Total', ascending=False).drop(columns='Total')

topic_wide

df = topic_wide


# Combine AISI and National Semiconductor Technology Center into NIST
df['NIST'] = df['NIST'] + df['AISI'] + df['National Semiconductor Technology Center']

# Combine PTO and Patent and Trial Appeal Board into USPTO
df['USPTO'] = df['USPTO'] + df['PTO'] + df['Patent and Trial Appeal Board']

# Drop merged columns
df = df.drop(columns=['AISI', 'National Semiconductor Technology Center', 'PTO', 'Patent and Trial Appeal Board'])

# Melt to long format
df_long = df.melt(id_vars='Topic', var_name='Assignee', value_name='Count')
df_long = df_long[df_long['Count'] > 0]

# Calculate total counts for each topic
totals = df_long.groupby('Topic')['Count'].sum().reset_index()

# Reorder Topic by total count (largest to smallest)
topic_order = totals.sort_values('Count', ascending=False)['Topic']
df_long['Topic'] = pd.Categorical(df_long['Topic'], categories=topic_order, ordered=True)

# Create stacked bar chart
fig = px.bar(
  df_long,
  x='Topic',
  y='Count',
  title='Recommendation Count by Topic (Stacked by Assignee)',
  labels={'Count': 'Number of Recommendations'},
  # color_discrete_sequence=px.colors.qualitative.Set3
  template='ggplot2'
)

fig.update_layout(barmode='stack', xaxis_tickangle=-45)


# Add annotations
for i, row in totals.iterrows():
  fig.add_annotation(
   x=row['Topic'],
   y=row['Count'],
   text=str(row['Count']),
   showarrow=False,
   yshift=10,
   font=dict(size=12, color='black')
  )

display(fig)


In [59]:
fig = px.treemap(
    df_long,
    path=['Topic', 'Assignee'],
    values='Count',
    title='Recommendation Count by Topic and Assignee',
    color='Assignee',
    color_discrete_sequence=px.colors.qualitative.Set3_r
)



fig.show()

Most of the recommendations are assigned to NIST and have to do with standards and regulation.